# Web Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

````{warning}
If it says it cannot load one of the libraries, use pip inside your notebook to install, then restart your kernel (Kernel menu, choose restart)

```
pip install beautifulsoup4
```
````


We have that `read_html` can get content from an actual website, not a data file that is hosted somewhere on the internet, that takes tables on a website and returns a list of DataFrames.

## Everything is Data

For the purpose of this class, it is best to think of the content on a web page like a datastructure.  

![html anatomy](https://cdn2.hubspot.net/hub/53/file-1519188549-jpg/blog-files/webpage-setup.jpg)


![HTML tree structure](http://www.w3schools.com/js/pic_htmltree.gif)

there are tags `<>` that define the structure, and these can be further classified with `classes`

## Scraping a URI website


We're going to create a DataFrame about URI CS & Statistics Faculty.

from the [people page](https://web.uri.edu/cs/people/) of the department website.


We can inspect the page to check that it's well structured

```{warning}
With great power comes great responsibility.

- always check the [robots.txt](https://web.uri.edu/robots.txt)
- do not do things that the owner says not to do
- government websites are typically safe
```

We'lls ave the URL for easy use

In [2]:
cs_people_url = 'https://web.uri.edu/cs/people/'

Then we can use the `requests` library to make a call to the internet.  It actually gets back a [response object](https://requests.readthedocs.io/en/latest/api/#requests.Response) which has a lot of extra information.  For today we only need the `content` from the page which is an attrtibute of that object

In [3]:
cs_people_html = requests.get(cs_people_url).content

This is raw:

In [4]:
type(cs_people_html)

bytes

In [5]:
cs_people_html

b'\n<!DOCTYPE html>\n<html lang="en-US">\n\t\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<link rel="profile" href="http://gmpg.org/xfn/11">\n\n<title>People &#8211; Department of Computer Science and Statistics</title>\n\r\n\t  <meta name=\'robots\' content=\'max-image-preview:large\' />\n<link rel=\'dns-prefetch\' href=\'//s.w.org\' />\n<link rel="alternate" type="application/rss+xml" title="Department of Computer Science and Statistics &raquo; Feed" href="https://web.uri.edu/cs/feed/" />\n<link rel="alternate" type="application/rss+xml" title="Department of Computer Science and Statistics &raquo; Comments Feed" href="https://web.uri.edu/cs/comments/feed/" />\n\t\t<script type="text/javascript">\n\t\t\twindow._wpemojiSettings = {"baseUrl":"https:\\/\\/s.w.org\\/images\\/core\\/emoji\\/13.0.1\\/72x72\\/","ext":".png","svgUrl":"https:\\/\\/s.w.org\\/images\\/core\\/emoji\\/13.0.1\\/svg\\/","svgExt":".svg","source":{"concatemoji"

But we do not need to manually write search tools, that's what [`BeautifulSoup`](https://beautiful-soup-4.readthedocs.io/en/latest/) is for.

In [6]:
cs_people = BeautifulSoup(cs_people_html,'html.parser')

In this object we can use any tag from the file and get back the first instance

In [7]:
cs_people.a

<a class="skip-link screen-reader-text" href="#content">Skip to content</a>

In [8]:
cs_people.h3

<h3 class="p-name"><a href="https://web.uri.edu/cs/meet/marco-alvarez/">Marco Alvarez</a></h3>

More helpful is the `find_all` method we wnat to find all `div` tags that are "peopleitem" class. We decided this by inspecting the code on the website.

In [9]:
cs_people.find_all("div","peopleitem")

[<div class="peopleitem h-card has-thumbnail">
 <header>
 <div class="header">
 <figure>
 <a href="https://web.uri.edu/cs/meet/marco-alvarez/"><img alt="" class="u-photo wp-post-image" height="120" loading="lazy" src="https://web.uri.edu/cs/files/marco-alvarez.png" width="120"/></a>
 </figure>
 <h3 class="p-name"><a href="https://web.uri.edu/cs/meet/marco-alvarez/">Marco Alvarez</a></h3>
 </div>
 </header>
 <div class="inside">
 <p class="people-title p-job-title">Associate Professor | Director of Graduate Studies</p>
 <p class="people-department">Computer Science</p>
 <p class="people-misc"><span class="p-tel">401.874.5009</span> – <a class="u-email" href="mailto:malvarez@uri.edu">malvarez@uri.edu</a></p>
 <div style="clear:both;"></div>
 </div>
 </div>,
 <div class="peopleitem h-card">
 <header>
 <div class="header">
 <h3 class="p-name"><a href="https://web.uri.edu/cs/meet/samantha-armenti/">Samantha Armenti</a></h3>
 </div>
 </header>
 <div class="inside">
 <p class="people-title p-

this is a long, object and we can see it looks iterable (`[` at the start)

```{important}
answer to questions about searching [from the docs](https://beautiful-soup-4.readthedocs.io/en/latest/index.html?highlight=find_all#the-keyword-arguments)
```

We could search all tags for the class attribute like this:

In [10]:
cs_people.find_all(class="peopleitem")[0]

SyntaxError: invalid syntax (3969715587.py, line 1)

We can also look at only the first instance

In [11]:
cs_people.find_all("peopleitem")[0]

IndexError: list index out of range

We notice that the name is inside a `<h3>` tag with class `p-name` and then inside an a tag after that.  We also know from looking at the overall page that there are lots of other a tags, so we do not want to search all of those.

In [12]:
names = [n.a.string for n in cs_people.find_all("h3","p-name")]
names

['Marco Alvarez',
 'Samantha Armenti',
 'Sarah Brown',
 'Michael Conti',
 'Noah Daniels',
 'Lisa DiPippo',
 'Victor Fay-Wolfe',
 'Lutz Hamel',
 'Abdeltawab Hendawi',
 'Jean-Yves Hervé',
 'Natallia Katenka',
 'Soheyb Kouider',
 'Edmund Lamagna',
 'Indrani Mandal',
 'Gavino Puggioni',
 'Jonathan Schrader',
 'Krishna Venkatasubramanian',
 'Jing Wu',
 'Yichi Zhang',
 'Guangyu Zhu',
 'Ashley Buchanan',
 'Nina Kajiji',
 'Rachel Schwartz',
 'Ying Zhang']

How to pull out the titles for each person (eg Assitatn Teaching Professor, Associate Professor)

In [13]:
titles = [t.string for t in cs_people.find_all("p","people-title")]

In [14]:
titles

['Associate Professor | Director of Graduate Studies',
 'Assistant Teaching Professor',
 'Assistant Professor',
 'Assistant Teaching Professor',
 'Assistant Professor',
 'Professor | Chair',
 'Professor',
 'Associate Professor ',
 'Assistant Professor',
 'Associate Professor',
 'Associate Professor | Director of Undergraduate Studies',
 'Associate Teaching Professor',
 'Professor',
 'Assistant Teaching Professor',
 'Associate Professor |  Statistics Section Head | Director of Graduate Studies',
 'Assistant Teaching Professor',
 'Assistant Professor',
 'Associate Professor',
 'Assistant Professor ',
 'Assistant Professor',
 'Limited Joint Appointment',
 'Adjunct Associate Professor',
 'Assistant Professor – Limited Joint Appointment',
 'Assistant Professor – Limited Joint Appointment']

We can pull out two more things, the people-department indicates who is CS & who is Statistics.

In [15]:
disciplines = [d.string for d in cs_people.find_all("p",'people-department')]
emails = [e.string for e in cs_people.find_all("a",'u-email')]

In [16]:
css_fac_df = pd.DataFrame({'name':names, 'title':titles,'e-mails':emails, 'discipline':disciplines})
css_fac_df

,name,title,e-mails,discipline
0,Marco Alvarez,Associate Professor | Director of Graduate Stu...,malvarez@uri.edu,Computer Science
1,Samantha Armenti,Assistant Teaching Professor,sarmenti@uri.edu,Computer Science
2,Sarah Brown,Assistant Professor,brownsarahm@uri.edu,Computer Science
3,Michael Conti,Assistant Teaching Professor,michaelconti@uri.edu,Computer Science
4,Noah Daniels,Assistant Professor,noah_daniels@uri.edu,Computer Science
5,Lisa DiPippo,Professor | Chair,ldipippo@uri.edu,Computer Science
6,Victor Fay-Wolfe,Professor,vfaywolfe@uri.edu,Computer Science
7,Lutz Hamel,Associate Professor,lutzhamel@uri.edu,Computer Science
8,Abdeltawab Hendawi,Assistant Professor,hendawi@uri.edu,Data Science | Computer Science
9,Jean-Yves Hervé,Associate Professor,jyh@cs.uri.edu,Computer Science


```{admonition}

think ahead: how could we add people's office numbers to the datagrame that we
just made?

*hint* my office is listed on [the page linked to my name](https://web.uri.edu/cs/meet/sarah-brown/)
```

### Do you have any recommendations for learning html/css for the purpose of webscraping?

For webscraping only, careful examination of the page for patterns will be mostly enough. Rely on the nesting that the inspect or an IDE does. The following simple facts about website design will mostly be enough:
- tags are written like `<tagname>`
- classes are used to group similar tags and written like `class = list of classes separated by spaces`
- ids are used to "name" specific instances of tags like `id=section-qa`
- the other values inside tags are called attributes (class, id, href are attributes)
- the `<a>` tag is for links and it written like `<a href="url">link text</a>`

I will look for some examples, though.

## Is there a way to search just one person with that method?

We definitely can after pulling all the names, but the scraping is really for pulling the structure and reformatting batches of information more than finding a single thing. For finding one thing, there are better tools.

### what all the little commands in the inspection of a website are for?

### How do you know what to use to iterate through values in the HTML?

I knew I wanted the to look at each person, that was the information I wanted from the page, from visual inspection.  Then I inspected on one of the boxes about a person and noticed that they were a `div` tag with `peopleitem` class.  I actually decided just the way we did it in class.  

In the card for one person, I found each piece of information I wanted and then looked for the closest tag.  I also tried things that did not work before class to work out an example that would work, but whe you are learning new things, that is how it works.  Letting the computer tell you that you made a mistake is totally fine! Try things and experiment. Errors are okay.  

### How to use the find_all feature for just the class name?
use it as a keyword parameter like `class=`

### Can we get more practice webscraping and using the .find_all() function?

Try pulling additional information from this page (like the webpage for each person) and the information from that page.

### Will making a DataFrame via web scraping be use the DataFrame constructor, or is there more to it?

Use the constructor !